<a href="https://colab.research.google.com/github/akanksha0911/Spamfilterassign/blob/main/spamfilter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the libs

In [157]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string

Created a CSV file with all the given documents in assignment and read the file here

In [158]:
df = pd.read_csv("/content/drive/MyDrive/SpamDoc - Sheet1.csv")
df.head()

,text,spam
0,Free -Coupons for next movie. The above links ...,1
1,Free -Coupons for next movie. The above links ...,1
2,Our records indicate your Pension is under per...,1
3,"I know that's an incredible statement, but bea...",1
4,"Dear recipient, Avangar Technologies announces...",1


there are only Spams so the 1 is assigned.

In [159]:
df.shape

(6, 2)

In [160]:
df.columns

Index(['text', 'spam'], dtype='object')

In [161]:
df.drop_duplicates(inplace=True)
print(df.shape)

(5, 2)


In [162]:
# download the stopwords package
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

function to clean the text and return the tokens. The cleaning of the text can be done by first removing punctuations and then removing the useless words also known as stop words

In [163]:
#Tokenization (a list of tokens), will be used as the analyzer

def token(text):

  #1 Remove Punctuationa
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join(nopunc)

#2 Remove Stop Words
    clean = [word for word in nopunc.split() if word.lower() not in stopwords.words('english')]
    return clean


# to show the tokenization
df['text'].head().apply(token)

0    [Free, Coupons, next, movie, links, take, stra...
2    [records, indicate, Pension, performing, see, ...
3    [know, thats, incredible, statement, bear, exp...
4    [Dear, recipient, Avangar, Technologies, annou...
5    [Enter, win, 25000, get, Free, Hotel, Night, c...
Name: text, dtype: object

Feature Extraction

In [164]:
# Now convert the text into a matrix of token counts :

from sklearn.feature_extraction.text import CountVectorizer
vector = CountVectorizer(analyzer=token).fit_transform(df['text'])

In [165]:
vector.shape

(5, 197)

In [166]:
print(vector)

  (0, 22)	1
  (0, 14)	1
  (0, 131)	1
  (0, 128)	1
  (0, 120)	1
  (0, 178)	1
  (0, 176)	1
  (0, 144)	1
  (0, 171)	1
  (0, 110)	1
  (0, 167)	1
  (0, 136)	1
  (0, 53)	1
  (0, 47)	1
  (0, 187)	1
  (0, 25)	1
  (0, 42)	1
  (0, 8)	1
  (0, 188)	1
  (1, 167)	1
  (1, 155)	1
  (1, 109)	1
  (1, 36)	1
  (1, 148)	1
  (1, 102)	1
  :	:
  (4, 75)	1
  (4, 12)	1
  (4, 11)	1
  (4, 43)	1
  (4, 74)	1
  (4, 2)	1
  (4, 169)	1
  (4, 31)	1
  (4, 175)	1
  (4, 137)	1
  (4, 72)	1
  (4, 24)	1
  (4, 66)	1
  (4, 76)	1
  (4, 183)	1
  (4, 57)	1
  (4, 1)	1
  (4, 151)	1
  (4, 35)	1
  (4, 67)	1
  (4, 129)	1
  (4, 149)	1
  (4, 63)	1
  (4, 54)	1
  (4, 48)	1


Now we need to split the data into training and testing sets, and then we will use this one row of data for testing to make our prediction later on and test to see if the prediction matches with the actual value.

In [167]:
#split the data into 70% training and 30% testing
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(vector, df['spam'], test_size=0.30, random_state=0)
# To see the shape of the data
print(vector.shape)
print(xtrain.shape)
print(xtest.shape)


(5, 197)
(3, 197)
(2, 197)


In [168]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)

X_train_transformed = tfidf_transformer.fit(xtrain)

X_test_transformed = tfidf_transformer.fit(xtest)

In [169]:
print(X_train_transformed)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)


Now we need to create and train the Multinomial Naive Bayes classifier which is suitable for classification with discrete features.

In [170]:
# create and train the Naive Bayes Classifier
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB().fit(xtrain, ytrain)

**To see the classifiers prediction and actual values on the data set :**



In [171]:
print(classifier.predict(xtrain))
print(ytrain.values)

[1 1 1]
[1 1 1]


Now let’s see how well our model performed by evaluating the Naive Bayes classifier and the report, confusion matrix & accuracy score.



In [172]:
# Evaluating the model on the training data set
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtrain)
print(classification_report(ytrain, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytrain, pred))
print("Accuracy: \n", accuracy_score(ytrain, pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         3

    accuracy                           1.00         3
   macro avg       1.00      1.00      1.00         3
weighted avg       1.00      1.00      1.00         3


Confusion Matrix: 
 [[3]]
Accuracy: 
 1.0


In [173]:
#print the predictions
print(classifier.predict(xtest))
#print the actual values
print(ytest.values)

[1 1]
[1 1]


It has predicted on test data accurately , 1 = spam

let’s evaluate the model on the test data set 


In [174]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
pred = classifier.predict(xtest)
print(classification_report(ytest, pred))
print()
print("Confusion Matrix: \n", confusion_matrix(ytest, pred))
print("Accuracy: \n", accuracy_score(ytest, pred))

              precision    recall  f1-score   support

           1       1.00      1.00      1.00         2

    accuracy                           1.00         2
   macro avg       1.00      1.00      1.00         2
weighted avg       1.00      1.00      1.00         2


Confusion Matrix: 
 [[2]]
Accuracy: 
 1.0


The classifier accurately identified the email messages as spam or not spam with 100% accuracy on the test data.

